In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame()
leng = 10000
df['f1'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['f2'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['f3'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['f4'] = np.concatenate([np.random.normal(0.2, 0.5, leng), np.random.normal(1, 0.5, leng)])
df['label'] = np.concatenate([ np.zeros(leng), np.ones(leng),])

dff = df.sample(len(df))
ratio =0.8
length = int(len(dff) * ratio)
trainset = dff.iloc[length:]
valset = dff.iloc[:length]

In [3]:
y_train = trainset['label'].values
y_val = valset['label'].values

lgb_train = lgb.Dataset(trainset[['f1', 'f2', 'f3', 'f4']], y_train)
lgb_val = lgb.Dataset(valset[['f1', 'f2', 'f3', 'f4']], y_val )

In [4]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 3,
        'num_threads': 6, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':100,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.907521
[20]	valid_0's auc: 0.910371
[30]	valid_0's auc: 0.927849
[40]	valid_0's auc: 0.948537
[50]	valid_0's auc: 0.960094
[60]	valid_0's auc: 0.970319
[70]	valid_0's auc: 0.97522
[80]	valid_0's auc: 0.977918
[90]	valid_0's auc: 0.979791
[100]	valid_0's auc: 0.980737
[110]	valid_0's auc: 0.981642
[120]	valid_0's auc: 0.982304
[130]	valid_0's auc: 0.982853
[140]	valid_0's auc: 0.983065
[150]	valid_0's auc: 0.983073
[160]	valid_0's auc: 0.983257
[170]	valid_0's auc: 0.983279
[180]	valid_0's auc: 0.983379
[190]	valid_0's auc: 0.98375
[200]	valid_0's auc: 0.983931
[210]	valid_0's auc: 0.984188
[220]	valid_0's auc: 0.984266
[230]	valid_0's auc: 0.98431
[240]	valid_0's auc: 0.984286
[250]	valid_0's auc: 0.98429
[260]	valid_0's auc: 0.984355
[270]	valid_0's auc: 0.984406
[280]	valid_0's auc: 0.984524
[290]	valid_0's auc: 0.984544
[300]	valid_0's auc: 0.98456
[310]	valid_0's auc: 0.984647
[320]	valid_0's auc: 

In [5]:
model.best_iteration

402

In [6]:
model.save_model('gbm_test')

In [7]:
model1 = lgb.Booster(model_file='gbm_test')

In [13]:
aa = model1.predict(valset[['f1', 'f2', 'f3', 'f4']],num_iteration=230)

In [14]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, aa)

0.98430952319233644